In [1]:
# https://www.kaggle.com/mathormad/inceptionv3-baseline-lb-0-379/code

In [2]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io
from skimage.transform import resize
from imgaug import augmenters as iaa
from tqdm import tqdm
import PIL
from PIL import Image
import cv2
from sklearn.utils import class_weight, shuffle
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings("ignore")
SIZE = 299

In [3]:
# https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric/notebook
import tensorflow as tf

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1) + (1 - K.mean(f1))

In [4]:
# Load dataset info
path_to_train = '../data/train/'
data = pd.read_csv('../data/train.csv')

In [5]:
data.head()

,Id,Target
0,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0,16 0
1,000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,7 1 2 0
2,000a9596-bbc4-11e8-b2bc-ac1f6b6435d0,5
3,000c99ba-bba4-11e8-b2b9-ac1f6b6435d0,1
4,001838f8-bbca-11e8-b2bc-ac1f6b6435d0,18


In [6]:
train_dataset_info = []
for name, labels in zip(data['Id'], data['Target'].str.split(' ')):
    train_dataset_info.append({
        'path':os.path.join(path_to_train, name),
        'labels':np.array([int(label) for label in labels])})
train_dataset_info = np.array(train_dataset_info)

In [7]:
train_dataset_info

array([{'path': '../data/train/00070df0-bbc3-11e8-b2bc-ac1f6b6435d0', 'labels': array([16,  0])},
       {'path': '../data/train/000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0', 'labels': array([7, 1, 2, 0])},
       {'path': '../data/train/000a9596-bbc4-11e8-b2bc-ac1f6b6435d0', 'labels': array([5])},
       ...,
       {'path': '../data/train/fff189d8-bbab-11e8-b2ba-ac1f6b6435d0', 'labels': array([7])},
       {'path': '../data/train/fffdf7e0-bbc4-11e8-b2bc-ac1f6b6435d0', 'labels': array([25,  2, 21])},
       {'path': '../data/train/fffe0ffe-bbc0-11e8-b2bb-ac1f6b6435d0', 'labels': array([2, 0])}],
      dtype=object)

In [8]:
class data_generator:
    
    def create_train(dataset_info, batch_size, shape, augument=True):
        assert shape[2] == 3
        while True:
            dataset_info = shuffle(dataset_info)
            for start in range(0, len(dataset_info), batch_size):
                end = min(start + batch_size, len(dataset_info))
                batch_images = []
                X_train_batch = dataset_info[start:end]
                batch_labels = np.zeros((len(X_train_batch), 28))
                for i in range(len(X_train_batch)):
                    image = data_generator.load_image(
                        X_train_batch[i]['path'], shape)   
                    if augument:
                        image = data_generator.augment(image)
                    batch_images.append(image/255.)
                    batch_labels[i][X_train_batch[i]['labels']] = 1
                yield np.array(batch_images, np.float32), batch_labels

    def load_image(path, shape):
        image_red_ch = Image.open(path+'_red.png')
        image_yellow_ch = Image.open(path+'_yellow.png')
        image_green_ch = Image.open(path+'_green.png')
        image_blue_ch = Image.open(path+'_blue.png')
        image = np.stack((
            np.array(image_red_ch),
            np.array(image_green_ch), 
            np.array(image_blue_ch)), -1)
        w, h = 512, 512
        zero_data = np.zeros((h, w), dtype=np.uint8)
#         image2 = np.stack((
#             np.array(image_yellow_ch),
#             zero_data, zero_data), -1)
#         print(image1.shape, image2.shape)
#         image = np.vstack((image1, image2))
        image = cv2.resize(image, (shape[0], shape[1]))
        return image

    def augment(image):
        augment_img = iaa.Sequential([
            iaa.OneOf([
                iaa.Affine(rotate=0),
                iaa.Affine(rotate=90),
                iaa.Affine(rotate=180),
                iaa.Affine(rotate=270),
                iaa.Fliplr(0.5),
                iaa.Flipud(0.5),
            ])], random_order=True)

        image_aug = augment_img.augment_image(image)
        return image_aug


In [9]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D, BatchNormalization, Input, Conv2D
from keras.applications.inception_v3 import InceptionV3
from keras.callbacks import ModelCheckpoint
from keras import metrics
from keras.optimizers import Adam 
from keras import backend as K
import keras
from keras.models import Model

Using TensorFlow backend.


In [10]:
def create_model(input_shape, n_out):
    input_tensor = Input(shape=input_shape)
    base_model = InceptionV3(include_top=False,
                   weights='imagenet',
                   input_shape=input_shape)
    bn = BatchNormalization()(input_tensor)
    x = base_model(bn)
    x = Conv2D(32, kernel_size=(1,1), activation='relu')(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(n_out, activation='sigmoid')(x)
    model = Model(input_tensor, output)
    
    return model

In [11]:
import tensorflow as tf
from tensorflow.python.ops import array_ops

# https://github.com/ailias/Focal-Loss-implement-on-Tensorflow/blob/master/focal_loss.py
def focal_loss_org(prediction_tensor, target_tensor, weights=None, alpha=0.25, gamma=2):
    r"""Compute focal loss for predictions.
        Multi-labels Focal loss formula:
            FL = -alpha * (z-p)^gamma * log(p) -(1-alpha) * p^gamma * log(1-p)
                 ,which alpha = 0.25, gamma = 2, p = sigmoid(x), z = target_tensor.
    Args:
     prediction_tensor: A float tensor of shape [batch_size, num_anchors,
        num_classes] representing the predicted logits for each class
     target_tensor: A float tensor of shape [batch_size, num_anchors,
        num_classes] representing one-hot encoded classification targets
     weights: A float tensor of shape [batch_size, num_anchors]
     alpha: A scalar tensor for focal loss alpha hyper-parameter
     gamma: A scalar tensor for focal loss gamma hyper-parameter
    Returns:
        loss: A (scalar) tensor representing the value of the loss function
    """
    sigmoid_p = tf.nn.sigmoid(prediction_tensor)
    zeros = array_ops.zeros_like(sigmoid_p, dtype=sigmoid_p.dtype)
    
    # For poitive prediction, only need consider front part loss, back part is 0;
    # target_tensor > zeros <=> z=1, so poitive coefficient = z - p.
    pos_p_sub = array_ops.where(target_tensor > zeros, target_tensor - sigmoid_p, zeros)
    
    # For negative prediction, only need consider back part loss, front part is 0;
    # target_tensor > zeros <=> z=1, so negative coefficient = 0.
    neg_p_sub = array_ops.where(target_tensor > zeros, zeros, sigmoid_p)
    per_entry_cross_ent = - alpha * (pos_p_sub ** gamma) * tf.log(tf.clip_by_value(sigmoid_p, 1e-8, 1.0)) \
                          - (1 - alpha) * (neg_p_sub ** gamma) * tf.log(tf.clip_by_value(1.0 - sigmoid_p, 1e-8, 1.0))
    return tf.reduce_sum(per_entry_cross_ent)


In [12]:
def focal_loss(weights=None, alpha=0.25, gamma=2):
    def focal_loss_my(target_tensor, prediction_tensor, ):
        r"""Compute focal loss for predictions.
            Multi-labels Focal loss formula:
                FL = -alpha * (z-p)^gamma * log(p) -(1-alpha) * p^gamma * log(1-p)
                     ,which alpha = 0.25, gamma = 2, p = sigmoid(x), z = target_tensor.
        Args:
         prediction_tensor: A float tensor of shape [batch_size, num_anchors,
            num_classes] representing the predicted logits for each class
         target_tensor: A float tensor of shape [batch_size, num_anchors,
            num_classes] representing one-hot encoded classification targets
         weights: A float tensor of shape [batch_size, num_anchors]
         alpha: A scalar tensor for focal loss alpha hyper-parameter
         gamma: A scalar tensor for focal loss gamma hyper-parameter
        Returns:
            loss: A (scalar) tensor representing the value of the loss function
        """
        sigmoid_p = tf.nn.sigmoid(prediction_tensor)
        zeros = array_ops.zeros_like(sigmoid_p, dtype=sigmoid_p.dtype)

        # For poitive prediction, only need consider front part loss, back part is 0;
        # target_tensor > zeros <=> z=1, so poitive coefficient = z - p.
        pos_p_sub = array_ops.where(target_tensor > zeros, target_tensor - sigmoid_p, zeros)

        # For negative prediction, only need consider back part loss, front part is 0;
        # target_tensor > zeros <=> z=1, so negative coefficient = 0.
        neg_p_sub = array_ops.where(target_tensor > zeros, zeros, sigmoid_p)
        per_entry_cross_ent = - alpha * (pos_p_sub ** gamma) * tf.log(tf.clip_by_value(sigmoid_p, 1e-8, 1.0)) \
                              - (1 - alpha) * (neg_p_sub ** gamma) * tf.log(tf.clip_by_value(1.0 - sigmoid_p, 1e-8, 1.0))
        return tf.reduce_sum(per_entry_cross_ent)
#         return K.mean(K.binary_crossentropy(target_tensor, prediction_tensor), axis=-1) + tf.reduce_sum(per_entry_cross_ent)
    return focal_loss_my

In [13]:
def focal_loss_fixed(y_true, y_pred):
    gamma = 2.
    alpha = 0.25
    print(y_pred)
    print(y_true)
    pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
    pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

#     pt_1 = K.clip(pt_1, 1e-3, .999)
#     pt_0 = K.clip(pt_0, 1e-3, .999)

    return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))


In [14]:
# def focal_loss(gamma=2., alpha=.25):
#     def focal_loss_fixed(y_true, y_pred):
#         pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
#         pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

#         pt_1 = K.clip(pt_1, 1e-3, .999)
#         pt_0 = K.clip(pt_0, 1e-3, .999)

#         return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
#     return focal_loss_fixed

In [15]:
# create callbacks list
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split



In [16]:
train_dataset_info.shape

(31072,)

In [17]:

# split data into train, valid
indexes = np.arange(train_dataset_info.shape[0])
# np.random.shuffle(indexes)
# train_indexes, valid_indexes = train_test_split(indexes, test_size=0.15, random_state=8)
n_splits = 5
kf = KFold(n_splits=n_splits, random_state=42, shuffle=True)
submit = pd.read_csv('../data/sample_submission.csv')

# train_generator = data_generator.create_train(
#     train_dataset_info[train_indexes], batch_size, (SIZE,SIZE,3), augument=True)
# validation_generator = data_generator.create_train(
#     train_dataset_info[valid_indexes], 32, (SIZE,SIZE,3), augument=False)

oof_class_preds = np.zeros((train_dataset_info.shape[0], 28))
sub_class_preds = np.zeros((submit.shape[0], 28))

fold_ = 0
epochs = 10; batch_size = 16
for train_indexes, valid_indexes in kf.split(indexes):
    
    checkpoint = ModelCheckpoint('../cache/InceptionV3.h5', monitor='val_loss', verbose=1, 
                                 save_best_only=True, mode='min', save_weights_only = True)
    reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, 
                                       verbose=1, mode='auto', epsilon=0.0001)
    early = EarlyStopping(monitor="val_loss", 
                          mode="min", 
                          patience=15)
    callbacks_list = [checkpoint, early, reduceLROnPlat]


    _preds = []
    # create train and valid datagens
    train_generator = data_generator.create_train(
        train_dataset_info[train_indexes], batch_size, (SIZE,SIZE,3), augument=True)
    validation_generator = data_generator.create_train(
        train_dataset_info[valid_indexes], 32, (SIZE,SIZE,3), augument=False)

    # warm up model
    model = create_model(
        input_shape=(SIZE,SIZE,3), 
        n_out=28)

    for layer in model.layers:
        layer.trainable = False
    model.layers[-1].trainable = True
    model.layers[-2].trainable = True
    model.layers[-3].trainable = True
    model.layers[-4].trainable = True
    model.layers[-5].trainable = True
    model.layers[-6].trainable = True
    
    model.compile(
        loss=f1_loss, 
        optimizer=Adam(1e-03),
        metrics=[f1])
#     model.summary()

    model.fit_generator(
        train_generator,
        steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
        validation_data=validation_generator,
        validation_steps=np.ceil(float(len(valid_indexes)) / float(batch_size)),
        epochs=2, 
        verbose=1)
    
    # train all layers
    epochs=120
    for layer in model.layers:
        layer.trainable = True
    model.compile(loss=f1_loss,
                optimizer=Adam(lr=1e-4),
                metrics=[f1])
    model.fit_generator(
        train_generator,
        steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
        validation_data=validation_generator,
        validation_steps=np.ceil(float(len(valid_indexes)) / float(batch_size)),
        epochs=epochs, 
        verbose=1,
        callbacks=callbacks_list)
    
    model.fit_generator(
        train_generator,
        steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
        validation_data=validation_generator,
        validation_steps=np.ceil(float(len(valid_indexes)) / float(batch_size)),
        epochs=epochs, 
        verbose=1,
        callbacks=callbacks_list)
    
    for idx in tqdm(valid_indexes):
        item = train_dataset_info[idx]
        path = item['path']
        labels = item['labels']
        image = data_generator.load_image(path, (SIZE,SIZE,3))/255.
        score_predict = model.predict(image[np.newaxis])[0]
        oof_class_preds[idx] = score_predict
        np.save('../cache/oof_class_preds-13.npy', oof_class_preds)
    
    for idx, name in tqdm(enumerate(submit['Id'])):
        path = os.path.join('../data/test/', name)
        image = data_generator.load_image(path, (SIZE,SIZE,3))/255.
        score_predict = model.predict(image[np.newaxis])[0]
        sub_class_preds[idx] += score_predict
        np.save('../cache/sub_class_preds-13.npy', sub_class_preds)
    fold_ += 1
sub_class_preds /= n_splits

Epoch 1/2
1554/1554 [==============================] - 363s 234ms/step - loss: 1.1206 - f1: 0.0377 - val_loss: 1.1826 - val_f1: 0.0349
Epoch 2/2
1554/1554 [==============================] - 352s 227ms/step - loss: 1.1054 - f1: 0.0499 - val_loss: 1.2400 - val_f1: 0.0322
Epoch 1/120
1554/1554 [==============================] - 422s 271ms/step - loss: 1.0539 - f1: 0.1014 - val_loss: 1.0248 - val_f1: 0.1594

Epoch 00001: val_loss improved from inf to 1.02485, saving model to ../cache/InceptionV3.h5
Epoch 2/120
1554/1554 [==============================] - 400s 258ms/step - loss: 0.9784 - f1: 0.1705 - val_loss: 0.9364 - val_f1: 0.2308

Epoch 00002: val_loss improved from 1.02485 to 0.93639, saving model to ../cache/InceptionV3.h5
Epoch 3/120
1554/1554 [==============================] - 400s 258ms/step - loss: 0.9339 - f1: 0.2085 - val_loss: 0.8930 - val_f1: 0.2699

Epoch 00003: val_loss improved from 0.93639 to 0.89304, saving model to ../cache/InceptionV3.h5
Epoch 4/120
1554/1554 [=========

1554/1554 [==============================] - 396s 255ms/step - loss: 0.7612 - f1: 0.3220 - val_loss: 0.7410 - val_f1: 0.3753

Epoch 00037: val_loss did not improve from 0.73655

Epoch 00037: ReduceLROnPlateau reducing learning rate to 9.999999717180686e-11.
Epoch 38/120
1554/1554 [==============================] - 397s 256ms/step - loss: 0.7608 - f1: 0.3226 - val_loss: 0.7395 - val_f1: 0.3764

Epoch 00038: val_loss did not improve from 0.73655
Epoch 39/120
1554/1554 [==============================] - 399s 257ms/step - loss: 0.7587 - f1: 0.3238 - val_loss: 0.7384 - val_f1: 0.3787

Epoch 00039: val_loss did not improve from 0.73655
Epoch 40/120
1554/1554 [==============================] - 397s 256ms/step - loss: 0.7621 - f1: 0.3209 - val_loss: 0.7391 - val_f1: 0.3774

Epoch 00040: val_loss did not improve from 0.73655

Epoch 00040: ReduceLROnPlateau reducing learning rate to 9.99999943962493e-12.
Epoch 1/120
1554/1554 [==============================] - 399s 257ms/step - loss: 0.7601 - f1

  0%|          | 0/6215 [00:00<?, ?it/s]


Epoch 00024: val_loss did not improve from 0.73655

Epoch 00024: ReduceLROnPlateau reducing learning rate to 9.999998977483754e-17.


100%|██████████| 6215/6215 [05:51<00:00, 17.68it/s]
11702it [08:02, 24.26it/s]


...

Epoch 1/2
1554/1554 [==============================] - 368s 237ms/step - loss: 1.1202 - f1: 0.0386 - val_loss: 1.2982 - val_f1: 0.0319
Epoch 2/2
1554/1554 [==============================] - 347s 223ms/step - loss: 1.1061 - f1: 0.0487 - val_loss: 1.1658 - val_f1: 0.0317
Epoch 1/120
1554/1554 [==============================] - 430s 277ms/step - loss: 1.0582 - f1: 0.0981 - val_loss: 1.0155 - val_f1: 0.1488

Epoch 00001: val_loss improved from inf to 1.01546, saving model to ../cache/InceptionV3.h5
Epoch 2/120
1554/1554 [==============================] - 400s 257ms/step - loss: 0.9840 - f1: 0.1673 - val_loss: 0.9163 - val_f1: 0.2399

Epoch 00002: val_loss improved from 1.01546 to 0.91633, saving model to ../cache/InceptionV3.h5
Epoch 3/120
1554/1554 [==============================] - 399s 257ms/step - loss: 0.9369 - f1: 0.2062 - val_loss: 0.9298 - val_f1: 0.2405

Epoch 00003: val_loss did not improve from 0.91633
Epoch 4/120
1554/1554 [==============================] - 398s 256ms/step - lo

1554/1554 [==============================] - 403s 260ms/step - loss: 0.7711 - f1: 0.3161 - val_loss: 0.7410 - val_f1: 0.3752

Epoch 00037: val_loss did not improve from 0.73782
Epoch 38/120
1554/1554 [==============================] - 403s 259ms/step - loss: 0.7705 - f1: 0.3156 - val_loss: 0.7431 - val_f1: 0.3725

Epoch 00038: val_loss did not improve from 0.73782
Epoch 39/120
1554/1554 [==============================] - 403s 259ms/step - loss: 0.7690 - f1: 0.3173 - val_loss: 0.7400 - val_f1: 0.3762

Epoch 00039: val_loss did not improve from 0.73782

Epoch 00039: ReduceLROnPlateau reducing learning rate to 9.999999092680235e-13.
Epoch 40/120
1554/1554 [==============================] - 401s 258ms/step - loss: 0.7693 - f1: 0.3173 - val_loss: 0.7408 - val_f1: 0.3749

Epoch 00040: val_loss did not improve from 0.73782
Epoch 41/120
1554/1554 [==============================] - 404s 260ms/step - loss: 0.7700 - f1: 0.3172 - val_loss: 0.7371 - val_f1: 0.3793

Epoch 00041: val_loss improved fr

  0%|          | 0/6214 [00:00<?, ?it/s]


Epoch 00016: val_loss did not improve from 0.73619

Epoch 00016: ReduceLROnPlateau reducing learning rate to 9.999999682655225e-23.


100%|██████████| 6214/6214 [05:39<00:00, 18.28it/s]
11702it [08:49, 22.09it/s]


In [18]:
np.save('../cache/oof_class_preds-13.npy', oof_class_preds)
np.save('../cache/sub_class_preds-13.npy', sub_class_preds)

In [19]:
# oof_class_preds = np.zeros((train_dataset_info.shape[0], 28))
# sub_class_preds = np.zeros((submit.shape[0], 28))
# score_predict = model.predict(image[np.newaxis])[0]
# oof_class_preds[idx] = score_predict

In [20]:
# for train_index, test_index in kf.split(indexes):
# ...    print("TRAIN:", train_index, "TEST:", test_index)

In [21]:
sub_class_preds.shape

(11702, 28)

In [22]:
predicted = []
for line in tqdm(sub_class_preds):
    label_predict = np.arange(28)[line>=0.2]
    str_predict_label = ' '.join(str(l) for l in label_predict)
    predicted.append(str_predict_label)

100%|██████████| 11702/11702 [00:00<00:00, 89289.54it/s]


In [23]:
len(predicted)

11702

In [24]:
predicted

['2',
 '5 25',
 '0 5 25',
 '0 25',
 '7 25',
 '4',
 '4 25',
 '0 23 25',
 '0',
 '0 25',
 '17 18 19 25',
 '3 5',
 '0 2 25',
 '7 9 10 20',
 '23',
 '0 4 5 18 25',
 '2 14',
 '0 5',
 '14 21',
 '0 5',
 '6',
 '3 5 24',
 '0 2 11 25',
 '0',
 '0 4 25',
 '0 11 12 21 25 26',
 '0',
 '0',
 '0 25',
 '0',
 '0 21',
 '0 7 25',
 '14 16 17 18 21 25',
 '0 5 25',
 '0 7',
 '13',
 '0 13 25',
 '0 3',
 '0 21 25',
 '1',
 '0 16 17 21 25',
 '6 25',
 '0 21 25',
 '18 19 25',
 '0 16 25',
 '6',
 '0',
 '0 23',
 '6 11 23',
 '0',
 '0 16 17 25',
 '0 5',
 '8 20 23 24',
 '0 25',
 '3',
 '0 25',
 '0 25',
 '11 23',
 '0 25',
 '21 25',
 '2 21 22',
 '0 5 21',
 '0 14 16 21',
 '7 21 25',
 '23',
 '0 18 19 25',
 '3 6 21 25',
 '0 25',
 '0 16 25',
 '21',
 '2 3',
 '0 2',
 '14',
 '4',
 '0 21',
 '0',
 '0 2 4',
 '0 1',
 '0 25',
 '0 25',
 '6 23 25',
 '0 25',
 '0 21',
 '21 25',
 '17 18',
 '0 23 25',
 '20 23',
 '0 21',
 '14 16 25',
 '11 14',
 '0 25',
 '11 14',
 '23',
 '13',
 '0 16 17 25',
 '0 25',
 '7 17 18 25',
 '0 7 19 25',
 '24',
 '0 23 25',

In [25]:
submit['Predicted'] = predicted

In [26]:
# Create submit
# submit = pd.read_csv('../data/sample_submission.csv')
# predicted = []
# draw_predict = []
# # model.load_weights('../cache/InceptionV3.h5')
# for name in tqdm(submit['Id']):
#     path = os.path.join('../data/test/', name)
#     image = data_generator.load_image(path, (SIZE,SIZE,3))/255.
#     score_predict = model.predict(image[np.newaxis])[0]
#     draw_predict.append(score_predict)
#     label_predict = np.arange(28)[score_predict>=0.2]
#     str_predict_label = ' '.join(str(l) for l in label_predict)
#     predicted.append(str_predict_label)

# submit['Predicted'] = predicted
# np.save('../cache/draw_predict_InceptionV3-8.npy', score_predict)
# submit.to_csv('../submissions/submit_InceptionV3.csv', index=False)

In [27]:
submit.to_csv('../submissions/sub13-a.csv', index=False)

In [28]:
#https://stackoverflow.com/questions/1855095/how-to-create-a-zip-archive-of-a-directory
def backup_project_as_zip(project_dir, zip_file):
    assert(os.path.isdir(project_dir))
    assert(os.path.isdir(os.path.dirname(zip_file)))
    shutil.make_archive(zip_file.replace('.zip',''), 'zip', project_dir)
    pass

In [29]:
import datetime, shutil
now = datetime.datetime.now()
print(now)
PROJECT_PATH = '/home/watts/lal/Kaggle/kagglehp/scripts_nbs'
backup_project_as_zip(PROJECT_PATH, '../cache/code.scripts_nbs.%s.zip'%now)

2018-10-30 00:48:13.957997


In [32]:
!pip install -U kaggle

Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 61kB 183kB/s ta 0:00:01
Requirement already up-to-date: urllib3<1.23.0,>=1.15 in /home/watts/anaconda3/envs/hpg/lib/python3.6/site-packages (from kaggle)
Requirement already up-to-date: six>=1.10 in /home/watts/anaconda3/envs/hpg/lib/python3.6/site-packages (from kaggle)
    100% |████████████████████████████████| 153kB 545kB/s ta 0:00:01
    100% |████████████████████████████████| 235kB 1.7MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 764kB/s ta 0:00:01
Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 51kB 917kB/s ta 0:00:011
  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/4b/2a/0276479a4b3caeb8a8c1af2f8e4355746a97fab05a372e4a2c6a6b876165/idna-2.7-py2.py3-none-any.whl
Requirement already up-to-date: chardet<3.1.0,>=3.0.2

In [33]:
%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub13-a.csv -m ""

100%|████████████████████████████████████████| 492k/492k [00:13<00:00, 37.3kB/s]
Successfully submitted to Human Protein Atlas Image ClassificationCPU times: user 349 ms, sys: 224 ms, total: 572 ms
Wall time: 17.2 s


In [34]:
from time import sleep
sleep(10)
!kaggle competitions submissions -c human-protein-atlas-image-classification

fileName      date                 description  status    publicScore  privateScore  
------------  -------------------  -----------  --------  -----------  ------------  
sub13-a.csv   2018-10-29 19:20:40               complete  0.444        None          
sub12-d.csv   2018-10-26 02:09:32               complete  0.466        None          
sub12-h.csv   2018-10-26 02:07:56               complete  0.389        None          
sub12-g.csv   2018-10-25 00:55:10               complete  0.433        None          
sub12-c.csv   2018-10-25 00:45:32               complete  0.469        None          
sub12-bb.csv  2018-10-25 00:43:34               complete  0.466        None          
sub12-b.csv   2018-10-25 00:41:50               complete  0.457        None          
sub12-a.csv   2018-10-25 00:40:56               complete  0.449        None          
sub11-k.csv   2018-10-24 00:35:39               complete  0.346        None          
sub11-j.csv   2018-10-24 00:34:46               comple

In [35]:
predicted = []
for line in tqdm(sub_class_preds):
    label_predict = np.arange(28)[line>=0.25]
    str_predict_label = ' '.join(str(l) for l in label_predict)
    predicted.append(str_predict_label)

100%|██████████| 11702/11702 [00:00<00:00, 90436.26it/s]


In [36]:
submit['Predicted'] = predicted
submit.to_csv('../submissions/sub13-b.csv', index=False)

In [37]:
import datetime, shutil
now = datetime.datetime.now()
print(now)
PROJECT_PATH = '/home/watts/lal/Kaggle/kagglehp/scripts_nbs'
backup_project_as_zip(PROJECT_PATH, '../cache/code.scripts_nbs.%s.zip'%now)

2018-10-30 00:51:23.536802


In [38]:
%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub13-b.csv -m ""

100%|████████████████████████████████████████| 486k/486k [00:13<00:00, 37.7kB/s]
Successfully submitted to Human Protein Atlas Image ClassificationCPU times: user 349 ms, sys: 199 ms, total: 548 ms
Wall time: 16.2 s


In [39]:
from time import sleep
sleep(10)
!kaggle competitions submissions -c human-protein-atlas-image-classification

fileName      date                 description  status    publicScore  privateScore  
------------  -------------------  -----------  --------  -----------  ------------  
sub13-b.csv   2018-10-29 19:21:53               complete  0.448        None          
sub13-a.csv   2018-10-29 19:20:40               complete  0.444        None          
sub12-d.csv   2018-10-26 02:09:32               complete  0.466        None          
sub12-h.csv   2018-10-26 02:07:56               complete  0.389        None          
sub12-g.csv   2018-10-25 00:55:10               complete  0.433        None          
sub12-c.csv   2018-10-25 00:45:32               complete  0.469        None          
sub12-bb.csv  2018-10-25 00:43:34               complete  0.466        None          
sub12-b.csv   2018-10-25 00:41:50               complete  0.457        None          
sub12-a.csv   2018-10-25 00:40:56               complete  0.449        None          
sub11-k.csv   2018-10-24 00:35:39               comple

In [42]:
d = {0.3:'bb', 0.35:'c', 0.4:'d', 0.45:'e', 0.5:'f'}

In [43]:
for alpha in [0.3, 0.35, 0.4, 0.45, 0.5]:
    predicted = []
    for line in tqdm(sub_class_preds):
        label_predict = np.arange(28)[line>=alpha]
        str_predict_label = ' '.join(str(l) for l in label_predict)
        predicted.append(str_predict_label)
    submit['Predicted'] = predicted
    name = '../submissions/sub13-' + d[alpha] + '.csv'
    print(name)
    submit.to_csv(name, index=False)

 99%|█████████▉| 11610/11702 [00:00<00:00, 116077.42it/s]

../submissions/sub13-bb.csv
../submissions/sub13-c.csv


  0%|          | 0/11702 [00:00<?, ?it/s]

../submissions/sub13-d.csv
../submissions/sub13-e.csv


100%|██████████| 11702/11702 [00:00<00:00, 117229.73it/s]

../submissions/sub13-f.csv


In [44]:
%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub13-bb.csv -m ""

100%|████████████████████████████████████████| 481k/481k [00:16<00:00, 30.2kB/s]
Successfully submitted to Human Protein Atlas Image ClassificationCPU times: user 427 ms, sys: 214 ms, total: 641 ms
Wall time: 19.6 s


In [45]:
from time import sleep
sleep(10)
!kaggle competitions submissions -c human-protein-atlas-image-classification

fileName      date                 description  status    publicScore  privateScore  
------------  -------------------  -----------  --------  -----------  ------------  
sub13-bb.csv  2018-10-29 19:23:01               complete  0.454        None          
sub13-b.csv   2018-10-29 19:21:53               complete  0.448        None          
sub13-a.csv   2018-10-29 19:20:40               complete  0.444        None          
sub12-d.csv   2018-10-26 02:09:32               complete  0.466        None          
sub12-h.csv   2018-10-26 02:07:56               complete  0.389        None          
sub12-g.csv   2018-10-25 00:55:10               complete  0.433        None          
sub12-c.csv   2018-10-25 00:45:32               complete  0.469        None          
sub12-bb.csv  2018-10-25 00:43:34               complete  0.466        None          
sub12-b.csv   2018-10-25 00:41:50               complete  0.457        None          
sub12-a.csv   2018-10-25 00:40:56               comple

In [46]:
%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub13-c.csv -m ""
from time import sleep
sleep(10)
!kaggle competitions submissions -c human-protein-atlas-image-classification

100%|████████████████████████████████████████| 477k/477k [00:08<00:00, 55.1kB/s]
Successfully submitted to Human Protein Atlas Image ClassificationfileName      date                 description  status    publicScore  privateScore  
------------  -------------------  -----------  --------  -----------  ------------  
sub13-c.csv   2018-10-29 19:24:00               complete  0.459        None          
sub13-bb.csv  2018-10-29 19:23:01               complete  0.454        None          
sub13-b.csv   2018-10-29 19:21:53               complete  0.448        None          
sub13-a.csv   2018-10-29 19:20:40               complete  0.444        None          
sub12-d.csv   2018-10-26 02:09:32               complete  0.466        None          
sub12-h.csv   2018-10-26 02:07:56               complete  0.389        None          
sub12-g.csv   2018-10-25 00:55:10               complete  0.433        None          
sub12-c.csv   2018-10-25 00:45:32               complete  0.469        None    